# Minimise Cost with Tariff
### Select House

In [3]:
from os import path
from os import getcwd, listdir
path_to_data = path.join('..', 'data', 'houses')
houses = listdir(path_to_data)
for h in houses: print(path.splitext(h)[0])

64613
69914
70177
70210
70243
70277
91341
91910
91911
91912
97245


In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from thesisInitial.cleaning_data import extract_data_from_house, \
    extract_rated_power, extract_average_time_on, apply_new_load, generate_load
import seaborn as sns
import thesisInitial.tariff as tf
import time

sns.set()
house_id = 64613
load_type = 'hot_water'

### Importing Data and cleaning up

In [5]:
data = extract_data_from_house(house_id, load_type)
rated_power = extract_rated_power(data.load)
average_duration = extract_average_time_on(data.load)
data.head()

,load,PV,AC,total_use,grid_import,grid_export
reading_datetime,,,,,,
2019-01-01 00:00:00,0.0194,-0.0733,70.0850,70.0311,70.1044,0.0
2019-01-01 00:05:00,0.0197,-0.0736,69.2019,69.1480,69.2216,0.0
2019-01-01 00:10:00,0.0192,-0.0728,66.6908,66.6372,66.7100,0.0
2019-01-01 00:15:00,0.0194,-0.0733,68.1369,68.0830,68.1563,0.0
2019-01-01 00:20:00,0.0225,-0.0742,81.0847,81.0330,81.1072,0.0


### Tariff Choices
There are 4 Tariff Categories Currently
<li> flat
<li> Time of Use (TOU)
<li> Flat with controlled Load
<li> TOU with controlled load

In [6]:
tariff_type = 'TOU'
tariffs = tf.read_tariffs()

tariff = None
for t in tariffs:
    if t.tariff_type == tariff_type:
        tariff = t

print("Tariff Name: " + tariff.tariff_name)


Tariff Name: OriginGo


In [7]:
data['import_cost'] = tariff.get_cost(data['2019'][['grid_import', 'load']] / 1000)
data['export_revenue'] = (data['2019'].grid_export / 1000) * tariff.get_fit()
print('Import Cost' + " ${:.2f}".format(data.import_cost.sum() / 100))
print("Export Revenue" " ${:.2f}".format(data.export_revenue.sum() / 100))
print("Net Cost" + " ${:.2f}".format(data.import_cost.sum() / 100 - data.export_revenue.sum() / 100))

Import Cost $1306.76
Export Revenue $249.07
Net Cost $1057.69


#### Preload Possible Load Times Using Average Duration and Rated Power

In [9]:
index = data['2019-01-01'].index
possible_loads = pd.DataFrame(index=index)
# print(possible_loads.index)
start = time.time()
for i in range(int((24*60) / 5)):
    start_time = pd.Timestamp(possible_loads.index[0]) + pd.Timedelta(value=i * 5, unit='m')
    new_load, break_flag = generate_load(index, i, average_duration, rated_power)
    if break_flag:
        break
    new_load = new_load.rename(start_time.time())
    possible_loads = pd.concat([possible_loads, new_load], axis=1)
print(time.time() - start)
possible_loads.head()

0.31017231941223145


,00:00:00,00:05:00,00:10:00,00:15:00,00:20:00,00:25:00,00:30:00,00:35:00,00:40:00,00:45:00,...,21:40:00,21:45:00,21:50:00,21:55:00,22:00:00,22:05:00,22:10:00,22:15:00,22:20:00,22:25:00
reading_datetime,,,,,,,,,,,,,,,,,,,,,
2019-01-01 00:00:00,132.7443,0.0000,0.0000,0.0000,0.0000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-01-01 00:05:00,132.7443,132.7443,0.0000,0.0000,0.0000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-01-01 00:10:00,132.7443,132.7443,132.7443,0.0000,0.0000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-01-01 00:15:00,132.7443,132.7443,132.7443,132.7443,0.0000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-01-01 00:20:00,132.7443,132.7443,132.7443,132.7443,132.7443,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### Preload Possible Load Times Using Average Duration and Rated Power

In [ ]:
index = data['2019-01-01'].index
possible_loads = pd.DataFrame(index=index)
# print(possible_loads.index)
start = time.time()
for i in range(int((24*60) / 5)):
    start_time = pd.Timestamp(possible_loads.index[0]) + pd.Timedelta(value=i * 5, unit='m')
    new_load, break_flag = generate_load(index, i, average_duration, rated_power)
    if break_flag:
        break
    new_load = new_load.rename(start_time.time())
    possible_loads = pd.concat([possible_loads, new_load], axis=1)
print(time.time() - start)
possible_loads.head()

### Minimising Cost Loop
Loops over all the possible load start times and returns the load and cost of the minimum load start time

In [24]:
# Date Range to Minimise Cost
date_range = '2019-02'

days_in_set = data[date_range].index.date
days_in_set = np.unique(days_in_set)
min_cost_load = pd.Series(dtype='float64')
start = time.time()
for day in days_in_set:
    date = str(day)
    print(date)
    total_use_without_load = data[date].total_use - data[date].load
    pv_day = data.PV[date]
    grid_import_cost = tariff.get_cost(data[date][['grid_import', 'load']] / 1000).sum()
    grid_export_revenue = data[date]['grid_export'] / 1000 * tariff.get_fit()
    # min_cost = grid_import_cost.sum() - grid_export_revenue.sum()
    min_cost = 10000
    max_time = 0
    max_load = data[date].load
    index = pd.date_range(pd.to_datetime(date+" 00:00:00"), periods=288, freq='5T')
    tariff_series = tariff.get_tariff_from_time_index(index.hour).astype(float)
    possible_loads.index = possible_loads.index.map(lambda x : x.replace(year=day.year, month=day.month, day=day.day))
    for col in possible_loads.columns:
        new_load = possible_loads.loc[:,col]
        new_total_use = total_use_without_load + new_load
        new_grid_import = (new_total_use - pv_day).clip(lower=0)
        new_grid_export = ((new_total_use - pv_day) * -1).clip(lower=0)
        new_grid_import_cost = (new_grid_import / 1000) * tariff_series
        new_grid_export_revenue = (new_grid_export / 1000) * tariff.get_fit()
        new_cost_net = new_grid_import_cost.sum(axis=0) - new_grid_export_revenue.sum(axis=0)
        if new_cost_net < min_cost:
            min_cost = new_cost_net
            max_load = new_load
            max_time = col

    min_cost_load = pd.concat([min_cost_load, max_load], axis=0)
print(time.time() - start)
print('done')

2019-02-01
2019-02-02
2019-02-03
2019-02-04
2019-02-05
2019-02-06
2019-02-07
2019-02-08
2019-02-09
2019-02-10
2019-02-11
2019-02-12
2019-02-13
2019-02-14
2019-02-15
2019-02-16
2019-02-17
2019-02-18
2019-02-19
2019-02-20
2019-02-21
2019-02-22
2019-02-23
2019-02-24
2019-02-25
2019-02-26
2019-02-27
2019-02-28
13.435747861862183
done


### Makes a new Dataframe with the minimised load instead of the real load

In [ ]:
data_copy = data[date_range].copy(deep=True)
new_load = min_cost_load[~min_cost_load.index.duplicated(keep='last')]
new_load = new_load.rename('load')
new_total_use = (data_copy.total_use - data_copy.load + new_load).rename('total_use')
new_AC = (new_total_use - data_copy.PV).rename('AC')
new_grid_import = new_AC.clip(lower=0).rename('grid_import')
new_grid_export = (new_AC * -1).clip(lower=0).rename('grid_export')
new_data = pd.concat([new_load, data_copy.PV, new_AC, new_total_use, new_grid_import, new_grid_export], axis=1)

In [ ]:
date = '2019-01-30'
days_in_set = data[date_range].index.date
days_in_set = np.unique(days_in_set)
for day in days_in_set:
    date = str(day)
    fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(10,6))
    data[date].total_use.plot(ax=ax[0], alpha=0.5)
    data[date].load.clip(lower=0).plot.area(ax=ax[0])
    data[date].grid_export.plot.area(ax=ax[0], alpha=0.7)
    cost = tariff.get_cost(data[date][['grid_import', 'load']]/1000).sum() / 100
    revenue = ((data[date].grid_export / 1000) * tariff.get_fit()).sum() / 100
    ax[0].set_title('Cost $%.2f\nRevenue $%.2f\nTotal $%.2f' % (cost, revenue, cost-revenue))
    ax[0].set_ylim([0,250])
    new_data[date].total_use.plot(ax=ax[1], alpha=0.5)
    new_data[date].load.clip(lower=0).plot.area(ax=ax[1])
    new_data[date].grid_export.plot.area(ax=ax[1], alpha=0.7)
    cost = tariff.get_cost(new_data[date][['grid_import', 'load']]/1000).sum() / 100
    revenue = ((new_data[date].grid_export / 1000) * tariff.get_fit()).sum() / 100
    ax[1].set_title('Cost $%.2f\nRevenue $%.2f\nTotal $%.2f' % (cost, revenue, cost-revenue))
    ax[1].set_ylim([0,250])
    plt.show()

In [ ]:
print('Without Controlling Load')
house_consumption_total = data[date_range].total_use.sum() / 1000
pv_gen_total = data[date_range].PV.sum() / 1000
grid_import_total = data[date_range].grid_import.sum() / 1000
# grid_import_cost_total = data['2019-01'].grid
grid_export_total = data[date_range].grid_export.sum() / 1000
cost = tariff.get_cost(data[date_range][['grid_import', 'load']] / 1000)
revenue = data[date_range].grid_export / 1000 * tariff.get_fit()
cost_total = cost.sum() / 100
revenue_total = revenue.sum() / 100
# pv_utilisation_total = new_data['2019-01'].pv_utilisation.sum() /1000

# export = pv_net - pv_utilisation
# export_total = export.sum() / 1000

print("House Usage Total\t\t" + "{:.2f}".format(house_consumption_total) + "kWh")
print("Import total\t\t\t" + "{:.2f}".format(grid_import_total) + "kWh")
print("Export total\t\t\t" + "{:.2f}".format(grid_export_total) + "kWh")
print("Import Cost\t\t\t" + "${:.2f}".format(cost_total))
print("Export Revenue Cost\t\t" + "${:.2f}".format(revenue_total))
print("Net\t\t\t\t" + "${:.2f}".format(cost_total - revenue_total))
print()
print("PV total\t\t\t" + "{:.2f}".format(pv_gen_total) + "kWh")
# print("PV self consumed total\t\t" + "{:.2f}".format(pv_utilisation_total) + "kWh")
# print("PV exported\t\t\t" + "{:.2f}".format(export_total) + "kWh")
# print("PV utilisation\t\t\t" + "{:.2f}".format((pv_utilisation_total / pv_gen_total) * 100) + "%")


In [ ]:
print('With Controlling Load')
house_consumption_total = new_data[date_range].total_use.sum() / 1000
pv_gen_total = new_data[date_range].PV.sum() / 1000
grid_import_total = new_data[date_range].grid_import.sum() / 1000
grid_export_total = new_data[date_range].grid_export.sum() / 1000
cost = tariff.get_cost(new_data[date_range][['grid_import', 'load']] / 1000)
revenue = new_data[date_range].grid_export / 1000 * tariff.get_fit()
cost_total = cost.sum() / 100
revenue_total = revenue.sum() / 100
# pv_utilisation_total = new_data['2019-01'].pv_utilisation.sum() /1000

# export = pv_net - pv_utilisation
# export_total = export.sum() / 1000

print("House Usage Total\t\t" + "{:.2f}".format(house_consumption_total) + "kWh")
print("Import total\t\t\t" + "{:.2f}".format(grid_import_total) + "kWh")
print("Export total\t\t\t" + "{:.2f}".format(grid_export_total) + "kWh")
print("Import Cost\t\t\t" + "${:.2f}".format(cost_total))
print("Export Revenue Cost\t\t" + "${:.2f}".format(revenue_total))
print("Net\t\t\t\t" + "${:.2f}".format(cost_total - revenue_total))
print()
print("PV total\t\t\t" + "{:.2f}".format(pv_gen_total) + "kWh")
# print("PV self consumed total\t\t" + "{:.2f}".format(pv_utilisation_total) + "kWh")
# print("PV exported\t\t\t" + "{:.2f}".format(export_total) + "kWh")
# print("PV utilisation\t\t\t" + "{:.2f}".format((pv_utilisation_total / pv_gen_total) * 100) + "%")